In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import xgboost as xgb

# Data preparation

* Fill missing values with zeros.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv")

df.head(5)

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [3]:
df.isnull().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

df.columns = df.columns.str.lower().str.replace(' ', '_')

df.fillna(0, inplace=True)


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

y_train = df_train['fuel_efficiency_mpg'].values
y_val = df_val['fuel_efficiency_mpg'].values
y_test = df_test['fuel_efficiency_mpg'].values

del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

In [5]:
df.isnull().sum()

engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

In [8]:
columns = df.columns

categorical_columns = list(set(df.dtypes[(df.dtypes == 'object') & (df.columns != 'fuel_efficiency_mpg')].index) & set(columns))
numerical_columns = list(set(df.dtypes[(df.dtypes != 'object') & (df.columns != 'fuel_efficiency_mpg')].index) & set(columns))

In [9]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=True)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

# Q1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.

In [10]:
from sklearn.tree import DecisionTreeRegressor, export_text

model = DecisionTreeRegressor(max_depth=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)


print(export_text(model, feature_names=list(dv.get_feature_names_out())))


|--- vehicle_weight <= 3022.11
|   |--- value: [16.88]
|--- vehicle_weight >  3022.11
|   |--- value: [12.94]



# Q2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)

In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

np.sqrt(np.mean((y_val-y_pred)**2))

np.float64(0.4586615458484907)

# Q3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.

In [12]:
for n in range(10, 201, 10):
    model = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    print(f"{n}: RMSE = {np.sqrt(np.mean((y_val-y_pred)**2)):0.3f}")

10: RMSE = 0.459
20: RMSE = 0.454
30: RMSE = 0.451
40: RMSE = 0.448
50: RMSE = 0.446
60: RMSE = 0.445
70: RMSE = 0.445
80: RMSE = 0.445
90: RMSE = 0.445
100: RMSE = 0.445
110: RMSE = 0.444
120: RMSE = 0.444
130: RMSE = 0.444
140: RMSE = 0.444
150: RMSE = 0.443
160: RMSE = 0.443
170: RMSE = 0.443
180: RMSE = 0.443
190: RMSE = 0.443
200: RMSE = 0.443


# Q4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`

In [13]:
rmses = []

for d in 10, 15, 20, 25:
    rmse = []
    for n in range(10, 201, 10):
        model = RandomForestRegressor(n_estimators=n, random_state=1, max_depth=d, n_jobs=-1)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)

        print(f"d={d}, n={n}: RMSE = {np.sqrt(np.mean((y_val-y_pred)**2)):0.3f}")
        rmse.append(np.sqrt(np.mean((y_val-y_pred)**2)))

    rmses.append(rmse)

d=10, n=10: RMSE = 0.451
d=10, n=20: RMSE = 0.447
d=10, n=30: RMSE = 0.446
d=10, n=40: RMSE = 0.443
d=10, n=50: RMSE = 0.442
d=10, n=60: RMSE = 0.442
d=10, n=70: RMSE = 0.441
d=10, n=80: RMSE = 0.442
d=10, n=90: RMSE = 0.442
d=10, n=100: RMSE = 0.441
d=10, n=110: RMSE = 0.441
d=10, n=120: RMSE = 0.441
d=10, n=130: RMSE = 0.441
d=10, n=140: RMSE = 0.440
d=10, n=150: RMSE = 0.440
d=10, n=160: RMSE = 0.440
d=10, n=170: RMSE = 0.440
d=10, n=180: RMSE = 0.440
d=10, n=190: RMSE = 0.440
d=10, n=200: RMSE = 0.440
d=15, n=10: RMSE = 0.458
d=15, n=20: RMSE = 0.453
d=15, n=30: RMSE = 0.451
d=15, n=40: RMSE = 0.448
d=15, n=50: RMSE = 0.446
d=15, n=60: RMSE = 0.445
d=15, n=70: RMSE = 0.445
d=15, n=80: RMSE = 0.445
d=15, n=90: RMSE = 0.445
d=15, n=100: RMSE = 0.445
d=15, n=110: RMSE = 0.444
d=15, n=120: RMSE = 0.444
d=15, n=130: RMSE = 0.444
d=15, n=140: RMSE = 0.444
d=15, n=150: RMSE = 0.443
d=15, n=160: RMSE = 0.443
d=15, n=170: RMSE = 0.443
d=15, n=180: RMSE = 0.442
d=15, n=190: RMSE = 0.442
d=15

In [14]:
np.asarray(rmses).shape

(4, 20)

In [15]:
np.mean(np.asarray(rmses),axis=1)

array([0.4418793 , 0.44561629, 0.44567934, 0.4457025 ])

# Q5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model

In [16]:
model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1,  n_jobs=-1)
model.fit(X_train, y_train)

importances = model.feature_importances_
feature_names = dv.get_feature_names_out()

In [17]:
feature_importance_df = pd.DataFrame({'Feature': feature_names,'Importance': importances}).sort_values(by='Importance', ascending=False)

feature_importance_df.head(5)

,Feature,Importance
13,vehicle_weight,0.959153
6,horsepower,0.016066
0,acceleration,0.011490
3,engine_displacement,0.003279
7,model_year,0.003170


# Q6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

In [18]:
features = list(dv.get_feature_names_out())
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model =  xgb.train(xgb_params, dtrain, num_boost_round=200, verbose_eval=5, evals=watchlist)
y_pred = model.predict(dval)

np.sqrt(np.mean((y_pred - y_val)**2))

[0]	train-rmse:1.81393	val-rmse:1.85444
[5]	train-rmse:0.51381	val-rmse:0.55664
[10]	train-rmse:0.37115	val-rmse:0.43896
[15]	train-rmse:0.34666	val-rmse:0.43362
[20]	train-rmse:0.33553	val-rmse:0.43376
[25]	train-rmse:0.32268	val-rmse:0.43683
[30]	train-rmse:0.31475	val-rmse:0.43752
[35]	train-rmse:0.30960	val-rmse:0.43784
[40]	train-rmse:0.30202	val-rmse:0.43968
[45]	train-rmse:0.29126	val-rmse:0.44024
[50]	train-rmse:0.28456	val-rmse:0.44140
[55]	train-rmse:0.27618	val-rmse:0.44225
[60]	train-rmse:0.26768	val-rmse:0.44290
[65]	train-rmse:0.26174	val-rmse:0.44352
[70]	train-rmse:0.25489	val-rmse:0.44531
[75]	train-rmse:0.24792	val-rmse:0.44628
[80]	train-rmse:0.24254	val-rmse:0.44689
[85]	train-rmse:0.23644	val-rmse:0.44749
[90]	train-rmse:0.23193	val-rmse:0.44839
[95]	train-rmse:0.22475	val-rmse:0.44904
[100]	train-rmse:0.21754	val-rmse:0.45039
[105]	train-rmse:0.21323	val-rmse:0.45075
[110]	train-rmse:0.20766	val-rmse:0.45166
[115]	train-rmse:0.20189	val-rmse:0.45237
[120]	train-rm

np.float64(0.46203156380184496)

In [19]:
features = list(dv.get_feature_names_out())
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model =  xgb.train(xgb_params, dtrain, num_boost_round=200, verbose_eval=5, evals=watchlist)
y_pred = model.predict(dval)

np.sqrt(np.mean((y_pred - y_val)**2))

[0]	train-rmse:2.28944	val-rmse:2.34561


[5]	train-rmse:1.41247	val-rmse:1.44988
[10]	train-rmse:0.91008	val-rmse:0.94062
[15]	train-rmse:0.63402	val-rmse:0.66672
[20]	train-rmse:0.48983	val-rmse:0.53064
[25]	train-rmse:0.41881	val-rmse:0.46891
[30]	train-rmse:0.38342	val-rmse:0.44289
[35]	train-rmse:0.36435	val-rmse:0.43250
[40]	train-rmse:0.35343	val-rmse:0.42746
[45]	train-rmse:0.34621	val-rmse:0.42595
[50]	train-rmse:0.33998	val-rmse:0.42498
[55]	train-rmse:0.33480	val-rmse:0.42449
[60]	train-rmse:0.33054	val-rmse:0.42456
[65]	train-rmse:0.32602	val-rmse:0.42493
[70]	train-rmse:0.32202	val-rmse:0.42503
[75]	train-rmse:0.31895	val-rmse:0.42526
[80]	train-rmse:0.31667	val-rmse:0.42563
[85]	train-rmse:0.31440	val-rmse:0.42574
[90]	train-rmse:0.31059	val-rmse:0.42586
[95]	train-rmse:0.30625	val-rmse:0.42611
[100]	train-rmse:0.30364	val-rmse:0.42635
[105]	train-rmse:0.30092	val-rmse:0.42694
[110]	train-rmse:0.29764	val-rmse:0.42721
[115]	train-rmse:0.29561	val-rmse:0.42721
[120]	train-rmse:0.29287	val-rmse:0.42730
[125]	train-

np.float64(0.4315182981569345)